In [1]:
%reload_ext autoreload
%autoreload 2
%reload_ext notexbook
%texify

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import time
import sys
sys.setrecursionlimit(10000)
%matplotlib qt5

In [3]:
import cellbgnet
import cellbgnet.utils
from cellbgnet.datasets import DataSimulator
from cellbgnet.utils.hardware import cpu, gpu
from cellbgnet.model import CellBGModel

In [4]:
param_file = '../../cellbgnet/utils/reference_files/reference.yaml'
param = cellbgnet.utils.param_io.ParamHandling().load_params(param_file)

In [5]:
psf_params = param.PSF.to_dict()
simulation_params = param.Simulation.to_dict()
hardware_params = param.Hardware.to_dict()
eval_params = param.Evaluation.to_dict()
train_size = simulation_params['train_size']

In [6]:
model = cellbgnet.model.CellBGModel(param)

training sliding windows on camera chip:
Area num: 0, field_xy: [0, 127, 0, 127]
Area num: 1, field_xy: [114, 241, 0, 127]
Area num: 2, field_xy: [228, 355, 0, 127]
Area num: 3, field_xy: [342, 469, 0, 127]
Area num: 4, field_xy: [456, 583, 0, 127]
Area num: 5, field_xy: [570, 697, 0, 127]
Area num: 6, field_xy: [684, 811, 0, 127]
Area num: 7, field_xy: [798, 925, 0, 127]
Area num: 8, field_xy: [912, 1039, 0, 127]
Area num: 9, field_xy: [1026, 1153, 0, 127]
Area num: 10, field_xy: [1140, 1267, 0, 127]
Area num: 11, field_xy: [1174, 1301, 0, 127]
Area num: 12, field_xy: [0, 127, 114, 241]
Area num: 13, field_xy: [114, 241, 114, 241]
Area num: 14, field_xy: [228, 355, 114, 241]
Area num: 15, field_xy: [342, 469, 114, 241]
Area num: 16, field_xy: [456, 583, 114, 241]
Area num: 17, field_xy: [570, 697, 114, 241]
Area num: 18, field_xy: [684, 811, 114, 241]
Area num: 19, field_xy: [798, 925, 114, 241]
Area num: 20, field_xy: [912, 1039, 114, 241]
Area num: 21, field_xy: [1026, 1153, 114, 24

In [7]:
model.init_recorder()
model.init_eval_data()

Eval image generation: 100%|████████████████████████████████████████| 30/30 [00:07<00:00,  3.79it/s]



eval images shape: (30, 1041, 1302) contain 10000 molecules,


In [8]:
model.evaluation_params['eval_imgs'].shape

(30, 1041, 1302)

In [9]:
simulation_params

{'train_size': 128,
 'train_type': 'cells',
 'psf_intensity_normalization': True,
 'density': 0.003125,
 'non_cell_density': 10,
 'margin_empty': 0.05,
 'z_prior': [-1, 1],
 'survival_prob': None,
 'min_photon': 0.4,
 'camera': 'sCMOS',
 'camera_chip_size': [1041, 1302],
 'perlin_noise': False,
 'perlin_noise_factor': 0.2,
 'perlin_noise_res': 64,
 'baseline': 103.0,
 'bg_values': 115.0,
 'e_per_adu': 0.4,
 'qe': 0.95,
 'spurious_c': 0.0,
 'sig_read': '/mnt/sda1/SMLAT/data/real_data/chromosome_dots/pooled_8865/RN_e_venus.tiff',
 'robust_training': False,
 'cell_masks_dir': '/mnt/sda1/SMLAT/data/real_data/chromosome_dots/pooled_8865/phase_venus_mask/',
 'cell_masks_filetype': '.png',
 'edt_noise_map_path': '/mnt/sda1/SMLAT/data/real_data/chromosome_dots/pooled_8865/edt_noise.pkl',
 'min_edt': 0,
 'max_edt': 6}

In [10]:
start_time = time.time()
model.fit(batch_size=32, max_iters=20000, print_output=True, print_freq=100)
duration = time.time() - start_time

Started training .... 
 || Cost: 538.058 || Time Upd.: 417.3 ms  || BatchNr.: 100
 || Cost: 185.756 || Time Upd.: 401.1 ms  || BatchNr.: 200
 || Cost: 168.521 || Time Upd.: 379.6 ms  || BatchNr.: 300
 || Cost: 121.281 || Time Upd.: 378.6 ms  || BatchNr.: 400
 || Cost: 98.345 || Time Upd.: 376.3 ms  || BatchNr.: 500
 || Cost: 89.440 || Time Upd.: 378.3 ms  || BatchNr.: 600
 || Cost: 81.784 || Time Upd.: 375.6 ms  || BatchNr.: 700
 || Cost: 69.446 || Time Upd.: 378.3 ms  || BatchNr.: 800
 || Cost: 63.562 || Time Upd.: 382.2 ms  || BatchNr.: 900
 || Cost: 58.425 || Time Upd.: 377.2 ms  || BatchNr.: 1000
processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]

evaluation on 30 images, contain ground truth: 10000, preds:5293
FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 10000 ,preds: 5293
JoR: 0.849 || Eff_3d: 32.385 || Jaccard: 0.529 || Factor: 277.740 || RMSE_lat: 62.305 || RMSE_ax: 64.648 || R

processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]

evaluation on 30 images, contain ground truth: 10000, preds:7101
FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 10000 ,preds: 7101
JoR: 1.272 || Eff_3d: 50.560 || Jaccard: 0.708 || Factor: 319.311 || RMSE_lat: 55.654 || RMSE_ax: 42.239 || RMSE_x: 40.206 || RMSE_y: 38.482 || Cost: 39.874 || Recall: 0.709 || Precision: 0.998 || BatchNr.: 2300 || Cost: 39.874 || Time Upd.: 386.6 ms 
saving this model, eff_3d, rmse_lat, rmse_ax and BatchNr are : 50.560 || 55.654 || 42.239 || 2300


processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]

evaluation on 30 images, contain ground truth: 10000, preds:6686
FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 10000 ,preds: 6686
JoR: 1.260 || Eff_3d: 48.491 || Jaccard: 0.665 || Factor: 302.258 || RMSE_lat: 52.

processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]

evaluation on 30 images, contain ground truth: 10000, preds:8184
FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 10000 ,preds: 8184
JoR: 1.419 || Eff_3d: 54.849 || Jaccard: 0.794 || Factor: 360.410 || RMSE_lat: 55.941 || RMSE_ax: 45.398 || RMSE_x: 40.691 || RMSE_y: 38.388 || Cost: 36.649 || Recall: 0.805 || Precision: 0.983 || BatchNr.: 3600 || Cost: 36.649 || Time Upd.: 406.2 ms 
saving this model, eff_3d, rmse_lat, rmse_ax and BatchNr are : 54.849 || 55.941 || 45.398 || 3600


processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]

evaluation on 30 images, contain ground truth: 10000, preds:7652
FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 10000 ,preds: 7652
JoR: 1.380 || Eff_3d: 54.748 || Jaccard: 0.763 || Factor: 328.174 || RMSE_lat: 55.

processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]

evaluation on 30 images, contain ground truth: 10000, preds:6674
FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 10000 ,preds: 6674
JoR: 1.236 || Eff_3d: 49.015 || Jaccard: 0.667 || Factor: 289.605 || RMSE_lat: 53.964 || RMSE_ax: 38.916 || RMSE_x: 37.334 || RMSE_y: 38.964 || Cost: 34.094 || Recall: 0.667 || Precision: 1.000 || BatchNr.: 4900 || Cost: 34.094 || Time Upd.: 382.8 ms 
saving this model, eff_3d, rmse_lat, rmse_ax and BatchNr are : 49.015 || 53.964 || 38.916 || 4900


processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]

evaluation on 30 images, contain ground truth: 10000, preds:7230
FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 10000 ,preds: 7230
JoR: 1.383 || Eff_3d: 53.855 || Jaccard: 0.723 || Factor: 310.347 || RMSE_lat: 52.

processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]

evaluation on 30 images, contain ground truth: 10000, preds:7874
FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 10000 ,preds: 7874
JoR: 1.451 || Eff_3d: 56.663 || Jaccard: 0.787 || Factor: 330.475 || RMSE_lat: 54.233 || RMSE_ax: 37.545 || RMSE_x: 39.575 || RMSE_y: 37.081 || Cost: 32.872 || Recall: 0.787 || Precision: 1.000 || BatchNr.: 6200 || Cost: 32.872 || Time Upd.: 396.7 ms 
saving this model, eff_3d, rmse_lat, rmse_ax and BatchNr are : 56.663 || 54.233 || 37.545 || 6200


processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]

evaluation on 30 images, contain ground truth: 10000, preds:7548
FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 10000 ,preds: 7548
JoR: 1.386 || Eff_3d: 54.807 || Jaccard: 0.755 || Factor: 311.602 || RMSE_lat: 54.

...

processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]

evaluation on 30 images, contain ground truth: 10000, preds:7934
FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 10000 ,preds: 7934
JoR: 1.495 || Eff_3d: 57.849 || Jaccard: 0.793 || Factor: 320.431 || RMSE_lat: 53.060 || RMSE_ax: 35.819 || RMSE_x: 38.139 || RMSE_y: 36.889 || Cost: 28.564 || Recall: 0.793 || Precision: 1.000 || BatchNr.: 14000 || Cost: 28.564 || Time Upd.: 387.1 ms 
saving this model, eff_3d, rmse_lat, rmse_ax and BatchNr are : 57.849 || 53.060 || 35.819 || 14000


processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]

evaluation on 30 images, contain ground truth: 10000, preds:8083
FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 10000 ,preds: 8083
JoR: 1.508 || Eff_3d: 58.146 || Jaccard: 0.808 || Factor: 319.668 || RMSE_lat: 5

processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]

evaluation on 30 images, contain ground truth: 10000, preds:7851
FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 10000 ,preds: 7851
JoR: 1.489 || Eff_3d: 57.532 || Jaccard: 0.785 || Factor: 313.217 || RMSE_lat: 52.676 || RMSE_ax: 35.848 || RMSE_x: 37.691 || RMSE_y: 36.798 || Cost: 27.648 || Recall: 0.785 || Precision: 1.000 || BatchNr.: 15300 || Cost: 27.648 || Time Upd.: 384.5 ms 
saving this model, eff_3d, rmse_lat, rmse_ax and BatchNr are : 57.532 || 52.676 || 35.848 || 15300


processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]

evaluation on 30 images, contain ground truth: 10000, preds:8242
FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 10000 ,preds: 8242
JoR: 1.536 || Eff_3d: 59.480 || Jaccard: 0.823 || Factor: 337.736 || RMSE_lat: 5

processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]

evaluation on 30 images, contain ground truth: 10000, preds:8212
FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 10000 ,preds: 8212
JoR: 1.543 || Eff_3d: 59.303 || Jaccard: 0.821 || Factor: 335.520 || RMSE_lat: 53.208 || RMSE_ax: 35.638 || RMSE_x: 38.542 || RMSE_y: 36.683 || Cost: 27.761 || Recall: 0.821 || Precision: 1.000 || BatchNr.: 16600 || Cost: 27.761 || Time Upd.: 382.4 ms 
saving this model, eff_3d, rmse_lat, rmse_ax and BatchNr are : 59.303 || 53.208 || 35.638 || 16600


processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]

evaluation on 30 images, contain ground truth: 10000, preds:7983
FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 10000 ,preds: 7983
JoR: 1.524 || Eff_3d: 58.639 || Jaccard: 0.798 || Factor: 316.597 || RMSE_lat: 5

processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]

evaluation on 30 images, contain ground truth: 10000, preds:8178
FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 10000 ,preds: 8178
JoR: 1.517 || Eff_3d: 58.914 || Jaccard: 0.818 || Factor: 330.558 || RMSE_lat: 53.883 || RMSE_ax: 35.029 || RMSE_x: 38.912 || RMSE_y: 37.273 || Cost: 27.648 || Recall: 0.818 || Precision: 1.000 || BatchNr.: 17900 || Cost: 27.648 || Time Upd.: 385.7 ms 
saving this model, eff_3d, rmse_lat, rmse_ax and BatchNr are : 58.914 || 53.883 || 35.029 || 17900


processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]

evaluation on 30 images, contain ground truth: 10000, preds:7927
FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 10000 ,preds: 7927
JoR: 1.511 || Eff_3d: 58.557 || Jaccard: 0.793 || Factor: 314.890 || RMSE_lat: 5

processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]

evaluation on 30 images, contain ground truth: 10000, preds:7741
FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 10000 ,preds: 7741
JoR: 1.474 || Eff_3d: 57.359 || Jaccard: 0.774 || Factor: 311.566 || RMSE_lat: 52.502 || RMSE_ax: 33.517 || RMSE_x: 37.407 || RMSE_y: 36.839 || Cost: 27.962 || Recall: 0.774 || Precision: 1.000 || BatchNr.: 19200 || Cost: 27.962 || Time Upd.: 375.0 ms 
saving this model, eff_3d, rmse_lat, rmse_ax and BatchNr are : 57.359 || 52.502 || 33.517 || 19200


processing area:99/99, input field_xy:[1258 1301 1004 1043], use_coordconv:True, retain locs in area:[1278, 1301, 1024, 1043]

evaluation on 30 images, contain ground truth: 10000, preds:8353
FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 10000 ,preds: 8353
JoR: 1.553 || Eff_3d: 59.569 || Jaccard: 0.834 || Factor: 340.022 || RMSE_lat: 5

In [11]:
duration

10939.329552650452